In [ ]:
import os
import yaml
import optuna
import json
import numpy as np
import matplotlib.pyplot as plt

from src.save_load import buildTargetMetadata, saveTarget, loadAudit
from src.train_models import trainTargetModel, trainFbDTargetModel
from src.visualize_model import VisualizeModel
from src.utils import print_yaml, sigmoid_weigths
from LeakPro.leakpro.attacks.mia_attacks.lira import lira_vectorized
#from LeakPro.leakpro.attacks.mia_attacks.rmia import rmia_vectorized

from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend

In [ ]:
# ------------------------------ #
#   Load Target and SM signals   #
# ------------------------------ #
audit_signals_folder = "20251117_b2919d4746"
metadata, rescaled_target_logits, rescaled_shadow_model_logits, shadow_models_in_mask, target_in_mask, audit_data_indices = loadAudit(audit_signals_folder)

In [ ]:
# -------------------------- #
#   Calculate audit scores   #
# -------------------------- #
lira_audit_cfg = {
    "var_calculation": "individual_carlini",
    "online": True,
    "fix_var_threshold": 32
}

#rmia_audit_cfg = {
#    "var_calculation": "individual_carlini",
#    "online": True,
#    "fix_var_threshold": 32
#}

lira_scores = lira_vectorized(rescaled_shadow_model_logits, shadow_models_in_mask, rescaled_target_logits, var_calculation="individual_carlini", online=True, fix_var_threshold=32)
rmia_scores = None #rmia_vectorized(rescaled_shadow_model_logits, shadow_models_in_mask, rescaled_target_logits, ...)

print(f"LiRA_scores: {lira_scores[:10]}\n")
#print(f"RMIA scores: {rmia_scores[:10]}\n")

In [ ]:
# ----------------------------------- #
#   Load FbD study & Update config   #
# ---------------------------------- #
# Initial configuration taken partially from "Technical Report for the Forgotten-by-Design Project: Targeted Obfuscation for Machine Learning"
fbd_cfg = {
    "noise_std": 0.01,
    "scale_alpha": 2,
    "scale_beta": 2,
    "centrality": 0.5,
    "temperature": 0.1,
    "scale_lower": 0.0,
    "scale_upper": None,
    "normalize": True
}

# Load studies or run with config defined above
load_study = False
if load_study:
    study_folder = "..."
    print("Loading study is not implemented")
    # Implement study load

print("\n----------------- Forgotten by Design configuration -----------------")
print_yaml(fbd_cfg)

In [ ]:
# --------------------- #
#   Calculate weights   #
# --------------------- #
if lira_scores is not None:
    print("\n--------------- Calculating LiRA scores ---------------")
    #lira_scores = fbd_cfg["scale_alpha"]*lira_scores - fbd_cfg["scale_beta"]
    lira_scores = np.clip(lira_scores, fbd_cfg["scale_lower"], fbd_cfg["scale_upper"])
    
    # Calculate weights using sigmoid func
    centrality = fbd_cfg["centrality"]
    temperature = fbd_cfg["temperature"]

    lira_weights = sigmoid_weights(lira_scores, centrality, temperature)
    #lira_weights = np.exp(-lira_scores)

    if fbd_cfg["normalize"]:
        lira_weights = lira_weights * (len(lira_weights)/lira_weights.sum())
        
    print(f"Calculated weights for LiRA audit: {lira_weights[:10]}")
    print(f"Shape of the weights for LiRA audit: {lira_weights.shape}")

if rmia_scores is not None:
    print("\n--------------- Calculating RMIA scores ---------------")
    rmia_scores = fbd_cfg["scale_alpha"]*rmia_scores - fbd_cfg["scale_beta"]
    rmia_scores = np.clip(rmia_scores, fbd_cfg["scale_lower"], fbd_cfg["scale_upper"])

    # Calculate weights using sigmoid func
    centrality = fbd_cfg["centrality"]
    temperature = fbd_cfg["temperature"]

    rmia_weights = sigmoid_weights(rmia_scores, centrality, temperature)
    #rmia_weights = np.exp(-rmia_scores)

    if fbd_cfg["normalize"]:
        rmia_weights = rmia_weights * (len(rmia_weights)/rmia_weights.sum())
        
    print(f"Calculated weights for RMIA audit: {rmia_weights[:10]}")
    print(f"Shape of the weights for RMIA audit: {rmia_weights.shape}")

In [ ]:
# ------------------------- #
#   Histogram for weights   #
# ------------------------- #
plt.hist(lira_weights, bins=30)
plt.show()
#plt.hist(rmia_weights, bins=30)
#plt.show()

In [ ]:
# --------------------------------------- #
#   Load train parameters from metadata   #
# --------------------------------------- #
    
train_cfg = metadata["trainCfg"]["train"]
data_cfg = metadata["trainCfg"]["data"]
run_cfg = metadata["trainCfg"]["run"]
    
print(f"Training config: {train_cfg}")

In [ ]:
# ------------------- #
#   Prepare dataset   #
# ------------------- #
from src.dataset_handler import processDataset, loadDataset, get_weighted_dataloaders

print("-------------------- Data_cfg --------------------")
print_yaml(data_cfg)

batch_size = train_cfg["batch_size"]

trainset, testset = loadDataset(data_cfg)

# Will split the dataset to use the same in indices as the baseline target model
train_dataset, test_dataset, train_indices, test_indices = processDataset(data_cfg, trainset, testset, in_indices_mask=target_in_mask)

# Prepare loaders with the lira weights
lira_train_loader, lira_test_loader = get_weighted_dataloaders(batch_size, train_dataset, test_dataset, lira_weights)

# Prepare loaders with the rmia weights
#rmia_train_loader, rmia_test_loader = get_weighted_dataloaders(batch_size, train_dataset, test_dataset, rmia_weights)

In [ ]:
# -------------------------- #
#   Train FbD Target model   #
# -------------------------- #
# FbD target model with weights calculated from the lira score
lira_model, lira_train_result, lira_test_result = trainFbDTargetModel(metadata["trainCfg"], lira_train_loader, lira_test_loader, train_indices, test_indices, fbd_cfg, "lira")
if(lira_train_result != None and lira_test_result != None):
    VisualizeModel().visualize(lira_train_result, lira_test_result)
    
